## Deliverable 2. Create a Customer Travel Destinations Map.

In [1]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps
import numpy as np
# Import API key
from config import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [2]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("../Weather_Database/WeatherPy_database.csv")
city_data_df.head()

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Punta Arenas,CL,-53.15,-70.92,37.4,86,100,8.05,overcast clouds
1,1,Avarua,CK,-21.21,-159.78,78.8,83,2,8.05,light rain
2,2,Dedza,MW,-14.38,34.33,46.6,96,55,5.01,broken clouds
3,3,Mahebourg,MU,-20.41,57.70,66.2,88,40,11.41,shower rain
4,4,Pafos,CY,34.77,32.42,73.4,73,0,5.82,clear sky


In [3]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))

What is the minimum temperature you would like for your trip? 75
What is the maximum temperature you would like for your trip? 90


In [4]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & (city_data_df["Max Temp"] >= min_temp)]
preferred_cities_df

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
1,1,Avarua,CK,-21.21,-159.78,78.80,83,2,8.05,light rain
14,14,Victoria,HK,22.29,114.16,82.40,74,40,10.29,scattered clouds
15,15,Santiago Tuxtla,MX,18.47,-95.30,75.22,93,100,4.79,moderate rain
17,17,Atuona,PF,-9.80,-139.03,78.94,74,5,21.77,clear sky
18,18,Phan Thiet,VN,10.93,108.10,88.12,61,40,5.95,scattered clouds
...,...,...,...,...,...,...,...,...,...,...
654,654,Kotturu,IN,14.82,76.22,80.26,73,14,2.26,few clouds
666,666,Padampur,IN,29.68,73.62,87.21,64,24,6.82,few clouds
670,670,Brigantine,US,39.41,-74.36,77.00,83,40,4.70,scattered clouds
674,674,Dingle,PH,11.00,122.67,89.02,67,54,6.15,light rain


In [5]:
# 4a. Determine if there are any empty rows.
preferred_cities_df.isnull().sum()

City_ID                0
City                   0
Country                0
Lat                    0
Lng                    0
Max Temp               0
Humidity               0
Cloudiness             0
Wind Speed             0
Current Description    0
dtype: int64

In [6]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
preferred_cities_df = preferred_cities_df.dropna()
preferred_cities_df.count()

City_ID                211
City                   211
Country                211
Lat                    211
Lng                    211
Max Temp               211
Humidity               211
Cloudiness             211
Wind Speed             211
Current Description    211
dtype: int64

In [7]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = preferred_cities_df[["City", "Country", "Max Temp", "Current Description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
1,Avarua,CK,78.80,light rain,-21.21,-159.78,
14,Victoria,HK,82.40,scattered clouds,22.29,114.16,
15,Santiago Tuxtla,MX,75.22,moderate rain,18.47,-95.30,
17,Atuona,PF,78.94,clear sky,-9.80,-139.03,
18,Phan Thiet,VN,88.12,scattered clouds,10.93,108.10,
24,Harda,IN,84.25,clear sky,22.33,77.10,
25,Nicoya,CR,75.56,light rain,10.15,-85.45,
26,Butaritari,KI,83.48,broken clouds,3.07,172.79,
30,Matara,LK,85.33,light rain,5.95,80.54,
31,Lazo,RU,79.20,broken clouds,43.38,133.90,


In [8]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

# 6b. Iterate through the hotel DataFrame.
for index, row in hotel_df.iterrows():  
    # 6c. Get latitude and longitude from DataFrame
    lat = row["Lat"]
    lng = row["Lng"]
    # Add the latitude and longitude to location key for the params dictionary.
    params["location"] = f"{lat},{lng}"
    
    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    # 6e. Make request and retrieve the JSON data from the search. 
    hotels = requests.get(base_url, params=params).json()
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except (IndexError):
        #print("Hotel not Found... skipping.")
        hotel_df.loc[index, "Hotel Name"] = np.nan

In [9]:
# 7. Drop the rows where there is no Hotel Name.
#hotel_df.count()
hotel_df.isnull().sum()
clean_hotel_df = hotel_df.dropna()
clean_hotel_df.count()

City                   197
Country                197
Max Temp               197
Current Description    197
Lat                    197
Lng                    197
Hotel Name             197
dtype: int64

In [10]:
# 8a. Create the output File (CSV)
output_data_file = "WeatherPy_vacation.csv"
# 8b. Export the City_Data into a csv
clean_hotel_df.to_csv(output_data_file, index_label="City_ID")

In [11]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Current Description</dt><dd>{Current Description}</dd>
<dt>Max Temp</dt><dd>{Max Temp} °F</dd>
</dl>
"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in clean_hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = clean_hotel_df[["Lat", "Lng"]]


In [12]:
# 11a. Add a marker layer for each city to the map. 
locations = clean_hotel_df[["Lat", "Lng"]]
max_temp = clean_hotel_df["Max Temp"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
heat_layer = gmaps.heatmap_layer(locations, weights=max_temp,dissipating=False,
             max_intensity=300, point_radius=4)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(heat_layer)
fig.add_layer(marker_layer)
# 11b. Display the figure
fig

Figure(layout=FigureLayout(height='420px'))